In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt
from pytz import timezone
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import TimeDistributed
from keras.utils.np_utils import to_categorical
from keras import optimizers
import dovahkiin as dk

Using TensorFlow backend.


In [3]:
# Set a random seed to reproduce
np.random.seed(7)

In [4]:
# Define Constatnts
prediction_length = 8
batch_size = 1
time_series_step = 32
features = 1

In [5]:
# Retrieve data source
dp = dk.DataParser()

In [6]:
X = dp.get_data("cu", split_threshold=1)

In [7]:
time_intervals = ["1T", "5T", "10T", "30T", "1H", "D", "W", "M"]

In [8]:
sgd = optimizers.SGD(lr=1e-8, decay=1e-9, momentum=0.9, nesterov=True, clipnorm=0.5)
adam = optimizers.Adam(lr=1e-4, clipnorm=1)
rmsprop = optimizers.rmsprop(lr=1e-4, decay=1e-8, clipnorm=1)
optimizer_list = [sgd, adam, rmsprop]

In [9]:
X = dp.get_data("cu", split_threshold="D")
X = pd.DataFrame((X["open"] + X["high"] + X["low"] + X["close"])/4, columns=["AvgPrice"])
res = sm.tsa.seasonal_decompose(X, model="additive", freq=(prediction_length - 1)*2, two_sided=True)
trend = res.trend.dropna()
noise = res.resid.dropna()
seasonal = res.seasonal.dropna()
trend_std_scaler = StandardScaler()

In [10]:
scaled_trend = trend_std_scaler.fit_transform(trend)
scaled_trend = pd.DataFrame(scaled_trend, index = trend.index)

### Data Preprocessing

In [11]:
def timeseries_to_supervised(raw_time_series, lag):
    p = {}
    for i in range(1, lag+1):
        p["{}".format(i)] = raw_time_series.shift(i).fillna(0) - raw_time_series
    
    supervised_data = pd.Panel(p)
    return supervised_data

def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

def create_supervised_X(raw_time_series, lag):
    supervised_X = timeseries_to_supervised(raw_time_series, lag)
    swaped_supervised_X = supervised_X.swapaxes(0, 1)
    return swaped_supervised_X

def create_future_y(time_series, future_steps):
    p = pd.DataFrame(index=time_series.index)
    for i in range(1, future_steps+1):
        p["{}".format(i)] = time_series.shift(-i).fillna(0) - time_series
    #p["0"] = raw_time_series
    # supervised_data = pd.DataFrame(p, index=time_series.index)
    return p

supervised_X = create_supervised_X(scaled_trend, lag=time_series_step)
supervised_X = supervised_X.fillna(0)
supervised_X_values = supervised_X.values

y = create_future_y(scaled_trend, prediction_length)

print("X shape:", supervised_X.shape)
print("y shape:", y.shape)

X shape: (3515, 32, 1)
y shape: (3515, 8)


In [12]:
assert(not (y == np.inf).any().any() or (y == -np.inf).any().any())
assert(not ((supervised_X == np.inf).any().any().any() or (supervised_X == -np.inf).any().any().any()))

### Model

In [13]:
model = Sequential()

# first layer of Market2Vec
model.add(Dense(32, 
                activation="softmax",
                batch_input_shape=(batch_size, time_series_step, 1)
               )
         )

# MultiLayer of LSTM to learn abstraction if any
model.add(
    LSTM(128,
         # batch_input_shape=(batch_size, time_series_step, features), 
         stateful=True,
         return_sequences=True,
         dropout=0.2
        )
)


model.add(
    LSTM(128,
         # batch_input_shape=(batch_size, time_series_step, features), 
         stateful=True, 
         return_sequences=True,
         dropout=0.2
        ))

model.add(
    LSTM(128, 
         stateful=True, 
         return_sequences=True,
         dropout=0.2
        ))

model.add(
    LSTM(128, 
         stateful=True, 
         return_sequences=True,
         dropout=0.2
        ))

model.add(
    LSTM(128, 
         stateful=True, 
         return_sequences=False,
         dropout=0.2
        ))
    
# # FullyConnected Network to final output
model.add(Dense(32, activation="softmax"))
model.add(Dense(32, activation="softmax"))
model.add(Dense(8, activation="linear"))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (1, 32, 32)               64        
_________________________________________________________________
lstm_1 (LSTM)                (1, 32, 128)              82432     
_________________________________________________________________
lstm_2 (LSTM)                (1, 32, 128)              131584    
_________________________________________________________________
lstm_3 (LSTM)                (1, 32, 128)              131584    
_________________________________________________________________
lstm_4 (LSTM)                (1, 32, 128)              131584    
_________________________________________________________________
lstm_5 (LSTM)                (1, 128)                  131584    
_________________________________________________________________
dense_2 (Dense)              (1, 32)                   4128      
__________

In [15]:
model.compile(loss="mse",
              optimizer=rmsprop)

### Training

In [18]:
history = 
    model.fit(
        supervised_X.values, 
        y.values,
        epochs=100,
        batch_size=batch_size,
        verbose=1,
        shuffle=False,
        validation_split=0.2
    )

Train on 2812 samples, validate on 703 samples
Epoch 1/100
2812/2812 [==============================] - 658s - loss: 0.0077 - val_loss: 0.0023
Epoch 2/100
2812/2812 [==============================] - 708s - loss: 0.0075 - val_loss: 0.0023
Epoch 3/100
2812/2812 [==============================] - 659s - loss: 0.0073 - val_loss: 0.0026
Epoch 4/100
2812/2812 [==============================] - 657s - loss: 0.0072 - val_loss: 0.0027
Epoch 5/100
2812/2812 [==============================] - 657s - loss: 0.0074 - val_loss: 0.0029
Epoch 6/100
2812/2812 [==============================] - 657s - loss: 0.0071 - val_loss: 0.0041
Epoch 7/100
2812/2812 [==============================] - 657s - loss: 0.0068 - val_loss: 0.0029
Epoch 8/100
2812/2812 [==============================] - 658s - loss: 0.0067 - val_loss: 0.0036
Epoch 9/100
2812/2812 [==============================] - 657s - loss: 0.0064 - val_loss: 0.0033
Epoch 10/100
2812/2812 [==============================] - 660s - loss: 0.0063 - val_loss:

2812/2812 [==============================] - 655s - loss: 0.0036 - val_loss: 0.0095
Epoch 86/100
2812/2812 [==============================] - 655s - loss: 0.0025 - val_loss: 0.0057
Epoch 87/100
2812/2812 [==============================] - 656s - loss: 0.0033 - val_loss: 0.0081
Epoch 88/100
2812/2812 [==============================] - 697s - loss: 0.0028 - val_loss: 0.0043
Epoch 89/100
2812/2812 [==============================] - 700s - loss: 0.0030 - val_loss: 0.0061
Epoch 90/100
2812/2812 [==============================] - 665s - loss: 0.0026 - val_loss: 0.0072
Epoch 91/100
2812/2812 [==============================] - 665s - loss: 0.0025 - val_loss: 0.0051
Epoch 92/100
2812/2812 [==============================] - 673s - loss: 0.0028 - val_loss: 0.0045
Epoch 93/100
2812/2812 [==============================] - 665s - loss: 0.0041 - val_loss: 0.0084
Epoch 94/100
2812/2812 [==============================] - 673s - loss: 0.0032 - val_loss: 0.0071
Epoch 95/100
2812/2812 [===================

In [19]:
model.save("Day_LSTM_Trend.h5")

In [ ]:
model.